In [0]:
!pip install kaggle
import os
import numpy as np
from google.colab import files
files.upload()

In [0]:
!mkdir /root/.kaggle 
!mv kaggle.json /root/.kaggle
!kaggle competitions download -c dogs-vs-cats

In [0]:
lap!unzip train.zip

In [0]:
!mkdir -p data/train/cats
!mkdir    data/train/dogs
!mkdir -p data/validation/cats
!mkdir    data/validation/dogs

In [0]:
for i in range(1000):
  os.rename(f"train/cat.{i}.jpg", f"data/train/cats/cat.{i + 1}.jpg")

for i in range(1000, 1400):
  os.rename(f"train/cat.{i}.jpg", f"data/validation/cats/cat.{i + 1 - 1000}.jpg")

In [0]:
for i in range(11100, 12100):
  os.rename(f"train/dog.{i}.jpg", f"data/train/dogs/dog.{i + 1 - 12500}.jpg")
  
for i in range(12100, 12500):
  os.rename(f"train/dog.{i}.jpg", f"data/validation/dogs/dog.{i + 1 - 12100}.jpg")

FileNotFoundError: ignored

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import optimizers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras import Model

In [0]:
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_psize=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
125/125 [==============================] - 19s 151ms/step - loss: 0.7457 - acc: 0.5390 - val_loss: 0.6757 - val_acc: 0.5938
Epoch 2/50
125/125 [==============================] - 16s 126ms/step - loss: 0.6784 - acc: 0.5920 - val_loss: 0.6437 - val_acc: 0.6375
Epoch 3/50
125/125 [==============================] - 16s 132ms/step - loss: 0.6428 - acc: 0.6370 - val_loss: 0.5964 - val_acc: 0.7087
Epoch 4/50
125/125 [==============================] - 15s 124ms/step - loss: 0.6130 - acc: 0.6680 - val_loss: 0.5823 - val_acc: 0.6775
Epoch 5/50
125/125 [==============================] - 15s 123ms/step - loss: 0.5875 - acc: 0.6925 - val_loss: 0.7534 - val_acc: 0.6075
Epoch 6/50
125/125 [==============================] - 15s 122ms/step - loss: 0.5885 - acc: 0.7105 - val_loss: 0.5640 - val_acc: 0.7113
Epoch 7/50
125/125 [==============================] - 15s 123ms/step - loss: 0.5740 - acc: 0.7110 - val_loss: 0.5338 - val_acc: 0.7175
Epoch 8

In [0]:
from keras import applications
import numpy as np
top_model_weights_path = 'bottleneck_fc_model.h5'

In [0]:
datagen = ImageDataGenerator(rescale=1. / 255)
generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

Found 2000 images belonging to 2 classes.


In [0]:
model = applications.VGG16(include_top=False, weights='imagenet')

bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

In [0]:
generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

Found 800 images belonging to 2 classes.


In [0]:
train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
train_labels = np.array(
     [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array(
     [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

top_model = Sequential()
top_model.add(Flatten(input_shape=train_data.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

top_model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
top_model.save_weights(top_model_weights_path)

In [0]:
top_model = Sequential()
top_model.add(Flatten(input_shape=train_data.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.load_weights(top_model_weights_path)

In [0]:
inp = Input((150, 150, 3))

vgg = model(inp)

output = top_model(vgg)

cnn = Model(inp, output)

In [0]:
for layer in cnn.layers[:25]:
    layer.trainable = False

In [0]:
cnn.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [0]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


In [0]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [0]:
cnn.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size)

Epoch 1/50
125/125 [==============================] - 56s 450ms/step - loss: 0.7155 - acc: 0.8970 - val_loss: 1.0061 - val_acc: 0.8708
Epoch 2/50
125/125 [==============================] - 55s 444ms/step - loss: 0.6444 - acc: 0.9100 - val_loss: 0.9984 - val_acc: 0.8716
Epoch 3/50
125/125 [==============================] - 54s 433ms/step - loss: 0.6260 - acc: 0.9015 - val_loss: 0.9996 - val_acc: 0.8716
Epoch 4/50
125/125 [==============================] - 54s 435ms/step - loss: 0.6722 - acc: 0.9075 - val_loss: 1.0035 - val_acc: 0.8709
Epoch 5/50
125/125 [==============================] - 55s 439ms/step - loss: 0.6355 - acc: 0.9090 - val_loss: 1.0030 - val_acc: 0.8709
Epoch 6/50
125/125 [==============================] - 53s 427ms/step - loss: 0.6709 - acc: 0.9060 - val_loss: 1.0029 - val_acc: 0.8716
Epoch 7/50
125/125 [==============================] - 54s 434ms/step - loss: 0.6488 - acc: 0.9060 - val_loss: 0.9989 - val_acc: 0.8714
Epoch 8/50
125/125 [==============================] - 5